In [1]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import statsmodels.stats.multitest as smm
from scipy import stats
import seaborn as sns

from Bio import Seq

pd.options.mode.chained_assignment = None

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

Change the following parameters to your output directory (where you want output files to be written), your metafile, and TNseq output. 

If these files aren't in the same directory as this notebook, you need use the full path. For example, the full path of 'barseq_output' would be '/usr2/people/clairedubin/barseq/barseq_output'

Also change the control and experimental temperatures if needed.

In [2]:
#where the BarSeq FASTQs are
FASTQ_directory = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024'


## Barseq metafile

You'll need to create your own metafile - easiest way is edit in Excel. A sample metafile is located at: /usr2/people/clairedubin/barseq/Kluyv_BarSeq_metadata.txt

    Fastq: path to barseq FASTQ for each competition
    SampleName: unique name for each competition, MUST include ctrl_temp or exp_temp as specified above
    DualIndex: index from sequencing (in read name lines of FASTQ and the sequencing info file that Adam sends)
    
    UsePrecounted: True if you've already run the Barseq counting script on a competition, False otherwise
    
    poolfile: path to Tnseq output file, should be the same for each competition
    output_dir: path to output directory
    minRandom: number of bases between start of read and DualIndex, usually in sequence of 1,2,3,4,1,2,3,4 for each sample
    maxRandom: same as minRandom
    
    BeforeBarcode: sequence preceding all barcodes
    AfterBarcode: sequence following all barcodes
    BarcodeLengths: lengths of barcodes to search for, with expected length of barcode first

    

In [3]:
!ls {FASTQ_directory}/*fastq

/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB001_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB002_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB003_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB004_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB005_S1_L001_R1_001.fastq
/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB006_S1_L001_R1_001.fastq


In [4]:
#format: sample_name, fastq_path, dual_index, offset
#add or delete more lines as needed

sample_info = [('KFRB001_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB001_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('KFRB002_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB002_S1_L001_R1_001.fastq', 'CGATGT', '2'), 
               ('KFRB003_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB003_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('KFRB004_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB004_S1_L001_R1_001.fastq', 'CGATGT', '2'),
               ('KFRB005_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB005_S1_L001_R1_001.fastq', 'ATCACG', '1'),
               ('KFRB006_1', '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB006_S1_L001_R1_001.fastq', 'CGATGT', '2'),
               ]

In [5]:
!head /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB001_S1_L001_R1_001.fastq

@M02248:14:000000000-DNH2T:1:1102:15643:1331 1:N:0:NTCACG
AGCACTAGTCGACCTGCAGCGTACGTATATGCGGTCCATGTTAGTAGAGACCTCGTGGACATCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAA
+
CCCCCFFFFFCBGGGGGGGGGEEFEGHHHHHHGGGGGHHHHHHHHHHHHHHGHHGHHGGHHHHGHHHHHGEGGGHHGHHHGHHHGFHHHHHHHHHGHHHHHGFEHFHHHGGHHHHGGGHGHHHHHHHCFGHHGGGG@A--
@M02248:14:000000000-DNH2T:1:1102:16005:1334 1:N:0:NTCACG
TTATTTCCTAGACAAATTAGAGCCAATACCATCAGCTTTACCGTCTTTCCAGAAATTGTTCCAAGTATCGGCAACAGCTTTATCAATACCATGAAAAATATCAACCACACCAGAAGCAGCATCAGTGACGACACTTGGAA
+
AAAAAFFFDFFFGGGG1BFGF1FFCGHHFFGFHHBGDHHGDGC0BAFDDGHFFG1FFFADEGHHHAEFFE/E/EFF0FFGGFHHHH1GFGHGFBGGHHHHDHFFBGCFFAEAGHHFFH0FGHHHBGFGH/FGGCFHGHFG
@M02248:14:000000000-DNH2T:1:1102:15502:1334 1:N:0:NTCACG
GGCACTAGTCGACCTGCAGCGTACGACGGTGCTCCAAGTGTGGGCAGAGACCTCGTGGACATCAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATCACGATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAATAT


In [6]:
#if there are any samples where you already counted barcodes, add the sample names here
precounted_samples = []

In [7]:
#these variables should be the same for each replicate

metafile_name = 'BarSeq_metafile_02_29_2024.txt'
logfile_name = 'BarSeq_counting_02_29_2024.log'
poolfile = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv'
output_dir = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024'
before_barcode = 'GTCGACCTGCAGCGTACG' #bottom strand sequence was used
#before_barcode = 'GATGTCCACGAGGTCTCT' what was there before 
after_barcode =  'AGAGACCTCGTGGACATC' #bottom strand sequence was used
#after_barcode = 'CGTACGCTGCAGGTCGAC'  what was there before 
barcode_lengths = '20,19,18,21'

In [8]:
metafile_columns = ['FileIndex','SampleName', 'Fastq', 'DualIndex', 'Poolfile', 'OutputDir', 'minRandom', 'maxRandom',
                   'BeforeBarcode', 'AfterBarcode', 'BarcodeLengths', 'UsePrecounted']

with open(metafile_name, 'w') as f:
    
    #write column names
    f.write('\t'.join(metafile_columns)+'\n')
    
    #write a line for each pool
    for sample_name, fastq, dual_index, offset in sample_info:
        
        #DUAL INDECES ARE BACKWARDS FROM WHAT THE FLOWCELL OUTPUT SAYS
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index[::-1], poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'False'])
        f.write(to_write+'\n')
        
    for precounted_sample_name in precounted_samples:
        
        to_write = '\t'.join([sample_name, sample_name, fastq, dual_index, poolfile, output_dir, offset, offset,
                             before_barcode, after_barcode, barcode_lengths, 'True'])




### Check metafile

In [9]:
pd.read_csv(metafile_name, sep='\t')

,FileIndex,SampleName,Fastq,DualIndex,Poolfile,OutputDir,minRandom,maxRandom,BeforeBarcode,AfterBarcode,BarcodeLengths,UsePrecounted
0,KFRB001_1,KFRB001_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB001_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
1,KFRB002_1,KFRB002_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB002_S1_L001_R1_001.fastq,TGTAGC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,2,2,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
2,KFRB003_1,KFRB003_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB003_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
3,KFRB004_1,KFRB004_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB004_S1_L001_R1_001.fastq,TGTAGC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,2,2,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
4,KFRB005_1,KFRB005_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB005_S1_L001_R1_001.fastq,GCACTA,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,1,1,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False
5,KFRB006_1,KFRB006_1,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/BarSeq_FASTQs_Feb_2024/KFRB006_S1_L001_R1_001.fastq,TGTAGC,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv,/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024,2,2,GTCGACCTGCAGCGTACG,AGAGACCTCGTGGACATC,"20,19,18,21",False


## Barseq counting

Might take a few hours!

In [10]:
!python3 /usr2/people/clairedubin/barseq/latest_pipeline/RBseq_Count_BarCodes_v2.py -m {metafile_name} -l {logfile_name}

2024-02-29 18:34:56 RBseq_Count_BarCodes.py
2024-02-29 18:34:56 Version: 1.1.4
2024-02-29 18:34:56 Release Date: July 16, 2020
2024-02-29 18:34:56 Options passed:  metafile:BarSeq_metafile_02_29_2024.txt  logFile:BarSeq_counting_02_29_2024.log  minQual:10  matchBefore:6  matchAfter:6  quietMode:False 
2024-02-29 18:34:56 Logging status updates in BarSeq_counting_02_29_2024.log
2024-02-29 18:34:56 Loading TnSeq library metadata from BarSeq_metafile_02_29_2024.txt
2024-02-29 18:34:56 Setting output directory as: /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp_repeat_02_29_2024 (from first line of metadata file)
2024-02-29 18:34:56 Loading mapped barcodes in mutant pool from: /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/Tnseq_11_04_2023_annotated_mod1.csv (from first line of metadata file)
2024-02-29 18:34:56 Read 4986 bar

2024-02-29 18:35:22   Total barcodes seen (incudes sequencing errors): 50950
2024-02-29 18:35:22   Matching barcodes to poolfile
2024-02-29 18:35:22     Number of barcodes from poolfile seen: 2992
2024-02-29 18:35:22     Reads with barcodes from poolfile: 566359
2024-02-29 18:35:22   Most abundant barcode: ACGGTGTTCCAAGTGTGGGC seen 124263 times.
2024-02-29 18:35:22   Number of reads that differ from this barcode by one base pair (likely sequencing errors):5571
2024-02-29 18:35:22   Estimated sequencing error rate for barcodes: 4.3%
2024-02-29 18:35:22   Barcodes seen once (highly inflated by sequencing errors): 37400
2024-02-29 18:35:22   Barcodes seen twice (slightly inflated by sequencing errors): 4176
2024-02-29 18:35:22   Barcodes seen three times or more: 9374
2024-02-29 18:35:22   Chao estimate of population size (ones^2/2*twos): 170000
2024-02-29 18:35:22 ---------------------
2024-02-29 18:35:22   Mapping reads from /usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Bars

### To convert the annotated file to csv and not to be separated by tab ##This section is not necessary here

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'OORB0010_1'
y_column = 'OORB0011_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('OORB0010_1 Abundance', fontsize=14)
plt.ylabel('OORB0011_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances (37 oC)', fontsize=16, color='purple')
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("fitness_of_TN_abundances_01_03_2024_37oC.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'OORB0010_1'
y_column = 'OORB0011_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('OORB010_1 Abundance', fontsize=14)
plt.ylabel('OORB011_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances (37 oC)', fontsize=16, color='purple')

# Adjust the x and y-axis scale ranges
plt.xlim(0, 15000)  # Adjust the range for the x-axis
plt.ylim(0, 15000)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)

plt.savefig("fitness_of_TN_abundances_03_01_2024_37oC_1a.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'OORB0012_1'
y_column = 'OORB0013_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('OORB012_1 Abundance', fontsize=14)
plt.ylabel('OORB013_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances (45 oC)', fontsize=16, color='purple')
#plt.grid(True)  # Optionally, add grid lines
#plt.savefig("fitness_of_TN_abundances_03_01_2024_45oC.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'OORB0012_1'
y_column = 'OORB0013_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('OORB012_1 Abundance', fontsize=14)
plt.ylabel('OORB013_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_45 oC', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 6000)  # Adjust the range for the x-axis
plt.ylim(0, 6000)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
plt.savefig("fitness_of_TN_abundances_03_01_2024_45oC_1a.png", dpi=600, bbox_inches='tight')
plt.show()

## to normalize the Barseq pool count file based on the number of reads obtained

#### normalization based on individual sample

In [ ]:
import pandas as pd

# Load the count file into a DataFrame
count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt'
df = pd.read_csv(count_file_path, sep='\t')

# Calculate the total counts for each replicate
df['Total_Count_OORB0010_1'] = df['OORB0010_1'].sum()
df['Total_Count_OORB0011_1'] = df['OORB0011_1'].sum()
df['Total_Count_OORB0012_1'] = df['OORB0012_1'].sum()
df['Total_Count_OORB0013_1'] = df['OORB0013_1'].sum()

# Normalize counts for each replicate by dividing by its own Total_Count
df['OORB0010_1'] = df['OORB0010_1'] / df['Total_Count_OORB0010_1']
df['OORB0011_1'] = df['OORB0011_1'] / df['Total_Count_OORB0011_1']
df['OORB0012_1'] = df['OORB0012_1'] / df['Total_Count_OORB0012_1']
df['OORB0013_1'] = df['OORB0013_1'] / df['Total_Count_OORB0013_1']

# Save the normalized count file
normalized_count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts_01_03_2023.csv'
df.to_csv(normalized_count_file_path, index=False)


In [ ]:
import pandas as pd

# Load the count file into a DataFrame
count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt'
df = pd.read_csv(count_file_path, sep='\t')

# Calculate the total counts for each replicate
df['Total_Count_OORB0010_1'] = df['OORB0010_1'].sum()
df['Total_Count_OORB0011_1'] = df['OORB0011_1'].sum()
df['Total_Count_OORB0012_1'] = df['OORB0012_1'].sum()
df['Total_Count_OORB0013_1'] = df['OORB0013_1'].sum()

# Normalize counts for each replicate by dividing by its own Total_Count
df['Normalized_OORB0010_1'] = (df['OORB0010_1'] / df['Total_Count_OORB0010_1']).round(3)
df['Normalized_OORB0011_1'] = (df['OORB0011_1'] / df['Total_Count_OORB0011_1']).round(3)
df['Normalized_OORB0012_1'] = (df['OORB0012_1'] / df['Total_Count_OORB0012_1']).round(3)
df['Normalized_OORB0013_1'] = (df['OORB0013_1'] / df['Total_Count_OORB0013_1']).round(3)

# Save the normalized count file
normalized_count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts_01_03_2023_mod.csv'
df.to_csv(normalized_count_file_path, index=False)


### this is the normalization that worked with the right result

In [ ]:
import pandas as pd

# Load the count file into a DataFrame
count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/poolCount.txt'
df = pd.read_csv(count_file_path, sep='\t')

# Calculate the total counts for each replicate
df['Total_Count_OORB0010_1'] = df['OORB0010_1'].sum()
df['Total_Count_OORB0011_1'] = df['OORB0011_1'].sum()
df['Total_Count_OORB0012_1'] = df['OORB0012_1'].sum()
df['Total_Count_OORB0013_1'] = df['OORB0013_1'].sum()

# Normalize counts for each replicate by dividing by its own Total_Count
df['Normalized_OORB0010_1'] = (df['OORB0010_1'] / df['Total_Count_OORB0010_1'])
df['Normalized_OORB0011_1'] = (df['OORB0011_1'] / df['Total_Count_OORB0011_1'])
df['Normalized_OORB0012_1'] = (df['OORB0012_1'] / df['Total_Count_OORB0012_1'])
df['Normalized_OORB0013_1'] = (df['OORB0013_1'] / df['Total_Count_OORB0013_1'])

# Save the normalized count file
normalized_count_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts_01_03_2023_mod1.csv'
df.to_csv(normalized_count_file_path, index=False)


##### To make the fitness abundance agaian using the normalized values

In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts_01_03_2023_mod1.csv'
df = pd.read_csv(csv_file_path)

# Save the DataFrame to a text file (txt)
txt_file_path = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts.txt'
df.to_csv(txt_file_path, sep='\t', index=False)


In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'Normalized_OORB0010_1'
y_column = 'Normalized_OORB0011_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_OORB0010_1 Abundance', fontsize=14)
plt.ylabel('Normalized_OORB0011_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances (37 oC)', fontsize=16, color='purple')
#plt.grid(True)  # Optionally, add grid lines
plt.savefig("fitness_of_TN_abundances_01_03_2024_37oC_Normalized.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'Normalized_OORB0012_1'
y_column = 'Normalized_OORB0013_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_OORB0012_1 Abundance', fontsize=14)
plt.ylabel('Normalized_OORB0013_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances (45 oC)', fontsize=16, color='purple')
#plt.grid(True)  # Optionally, add grid lines
plt.savefig("fitness_of_TN_abundances_01_03_2024_45oC_Normalized.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'Normalized_OORB0010_1'
y_column = 'Normalized_OORB0011_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_OORB0010_1 Abundance', fontsize=14)
plt.ylabel('Normalized_OORB0011_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_37 oC_normalized', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.05)  # Adjust the range for the x-axis
plt.ylim(0, 0.05)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_37oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Load the data from the CSV file into a DataFrame
df = pd.read_csv('/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/TNSeq_barseq_outputs_Temp2_12_31_2023/normalized_barseq_pool_counts.txt', sep='\t')

# Select the columns you want to use for the scatter plot
x_column = 'Normalized_OORB0012_1'
y_column = 'Normalized_OORB0013_1'

# Create a scatter plot using the selected columns
plt.scatter(df[x_column], df[y_column], color='black')
plt.xlabel('Normalized_OORB0012_1 Abundance', fontsize=14)
plt.ylabel('Normalized_OORB0013_1 Abundance', fontsize=14)
plt.title('Plot of TN_insertion abundances_45 oC_normalized', fontsize=16, color='purple')
# Adjust the x and y-axis scale ranges
plt.xlim(0, 0.05)  # Adjust the range for the x-axis
plt.ylim(0, 0.05)  # Adjust the range for the y-axis

# Optionally, add grid lines
#plt.grid(True)
#plt.grid(True)  # Optionally, add grid lines
plt.savefig("/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Barseq_Temp_study_11_04_2023/histogram_plots/fitness_of_TN_abundances_01_09_2024_45oC_Normalized_1.png", dpi=600, bbox_inches='tight')
plt.show()